In [18]:

def nextIndex(arr, index): #get value of next index
    next_index= 0
    previous = 0
    l = 0
    
    for i in range(index, len(arr)):
        if(arr[i]==0):    
            continue
        elif(arr[i] == previous):
            continue
        else: 
            previous = arr[i]
            if(l == 0):
                l = 1
                continue
            elif(l==1):
                return arr[i]
                



In [19]:
import torch
import random
import Levenshtein
from utils_common import count_parameters, get_padded_labels, decode_prediction, recognition_metrics
import numpy as np
# Swap function


def replace_pred(pred_seq, target):
    count = 0
    for i, p in enumerate(pred_seq):
        if(p != 0): 
            if(pred_seq[i] == pred_seq[i+1]):
                pred_seq[i] = target[count] 
            else:
                pred_seq[i] = target[count]
                count += 1
    return pred_seq

def create_new_timestep(arr):
    #arr = np.array([28, 19, 54, 21, 20])
    test_arr = np.zeros((31,), dtype=int)
    j = 0
    for i in range(len(test_arr)):
        if(i % 2 == 0):
            if(j < len(arr)):
                test_arr[i] = arr[j]
            j = j + 1

    # print(test_arr)
    return test_arr




def rectify_tazin(preds, labels, inv_grapheme_dict):
    
    _, pred = preds.max(2)
    pred_from_t = pred.transpose(
        1, 0).contiguous().detach().cpu().numpy()

    labels = torch.tensor(labels, dtype=torch.long)
    labels = labels.detach().numpy()
    for i in range(len(labels)):
        
        # Pre-process
        decoded, _ = decode_prediction(pred_from_t[i], inv_grapheme_dict)
        
        print(labels[i])
        print(type(labels[i]))

        Label_decoded, _ = decode_prediction(labels[i], inv_grapheme_dict, gt = True)
        _, decoded_pred = decode_prediction(decoded, inv_grapheme_dict)
        _, decoded_label = decode_prediction(
            labels[i], inv_grapheme_dict)

        
        
        if(decoded != Label_decoded):
                    


            if(len(ads_grapheme_extraction(decoded_label)) < len(ads_grapheme_extraction(decoded_pred))):
                print('prediction', pred_from_t[i])
                print('preds ', decoded)
                print("labels",Label_decoded)
                pred_from_t[i] = predGreaterLabel(pred_from_t[i], Label_decoded)
                print("--------------Recitified-----------")
                print(pred_from_t[i])
                print('\n')
              
 
            
                

                
     
    pred_from_t  = torch.from_numpy(pred_from_t)
    return pred_from_t

In [20]:
import torch
import random
import numpy
from utils_common import count_parameters, get_padded_labels, decode_prediction, recognition_metrics
import numpy as np

# swap function


def swap(a, b):
    temp = a
    a = b
    b = temp
    return a, b

# rectify function for training


def rectify(preds, target_timesteps, timesteps):
    preds = torch.transpose(preds, 0, 1)
    maxvalueindeces = torch.argmax(preds, dim=2)
    batch_size = 3
    
    
    for image in range(batch_size):

        for timestep in range(timesteps):
            

            maxpredindex = maxvalueindeces[image][timestep]
            maxtarindex = target_timesteps[image][timestep]

            # swap if max_grapheme_index is not equal to max_target_index
 
            preds[image][timestep][maxpredindex], preds[image][timestep][maxtarindex] = swap(
                preds[image][timestep][maxpredindex].item(), preds[image][timestep][maxtarindex].item())

    preds = torch.transpose(preds, 0, 1)
    return preds


In [21]:
from base64 import decode
import os
import random
import pickle
from tqdm import tqdm

import torch
from torch import optim
from torch.utils.data import DataLoader
from torch import quantization

from torchsummary import summary
from torchvision import transforms, datasets

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, f1_score

from datasets import SynthDataset
from utils_common import count_parameters, get_padded_labels, decode_prediction, recognition_metrics
from utils_train import preproc_synth_train_data, preproc_synth_valid_data
from model_vgg import get_crnn


# seeding for reproducability
random_seed = 33
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Path to dataset
DATA_PATH = "data/Bangla_writing_hnhtrd/"

# load grapheme dictionary of pre-trained model
with open("banglawriting_dict.pickle", 'rb') as handle:
    inv_grapheme_dict = pickle.load(handle)


# Preprocess the data and get grapheme dictionary and labels
inv_grapheme_dict, words_tr, labels_tr, lengths_tr = preproc_synth_train_data(
    os.path.join(DATA_PATH, "train_labels.txt"), representation='ads')
words_tr, labels_tr, lengths_tr = preproc_synth_valid_data(os.path.join(
    DATA_PATH, "train_labels.txt"), inv_grapheme_dict, representation='ads')
words_val, labels_val, lengths_val = preproc_synth_valid_data(os.path.join(
    DATA_PATH, "valid_labels.txt"), inv_grapheme_dict, representation='ads')

# Save grapheme dictionary
# https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
with open("./ckpts/inv_grapheme_dict_synth.pickle", 'wb') as handle:
    pickle.dump(inv_grapheme_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# print(inv_grapheme_dict)
# print(len(inv_grapheme_dict))


# Albumentations noise
data_transform = A.Compose([
    #A.imgaug.transforms.IAAPerspective (scale=(0.05, 0.05), keep_size=True, always_apply=False, p=0.5),
    #A.imgaug.transforms.IAAAffine (scale=1.0, translate_percent=None, translate_px=None, rotate=5.0, shear=5.0, order=1, cval=0, mode='reflect', always_apply=False, p=0.5),
    A.ElasticTransform(alpha=0.5, sigma=0, alpha_affine=0, p=0.3),
    A.augmentations.transforms.GaussNoise(var_limit=(
        120.0, 135.0), mean=0, always_apply=False, p=0.6),
    # A.imgaug.transforms.IAAAdditiveGaussianNoise(loc=1, scale=(2.55, 12.75), per_channel=False, always_apply=False, p=0.4),
    A.augmentations.transforms.MotionBlur(blur_limit=(3, 6), p=0.3),
    ToTensorV2(),
])

# Dataset
train_dataset = SynthDataset(os.path.join(
    DATA_PATH, "train_img"), transform=data_transform)
valid_dataset = SynthDataset(os.path.join(
    DATA_PATH, "val_img"), transform=data_transform)

num_train_samples = len(train_dataset)
num_valid_samples = len(valid_dataset)


# Creating data indices for training and validation splits
# validation_split = 0.2
# shuffle_dataset = True

# dataset_size = len(ocr_dataset)
# indices = list(range(dataset_size))
# split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.shuffle(indices)
# train_indices, val_indices = indices[split:], indices[:split]

# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)

# Batch Size variable
train_batch_s = 250
valid_batch_s = 250

train_loader = DataLoader(
    train_dataset, batch_size=train_batch_s, shuffle=True, num_workers=0)
validation_loader = DataLoader(
    valid_dataset, batch_size=valid_batch_s, shuffle=True, num_workers=0)

# Define model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = get_crnn(len(inv_grapheme_dict), qatconfig=None)
model = model.to(device)

# Teacher model
teacher_model = get_crnn(len(inv_grapheme_dict), qatconfig=None)
teacher_model = model.to(device)
teacher_model.load_state_dict(torch.load(
    'Teacher_epoch39.pth'.format(20), map_location=device))



###########
for name, param in model.named_parameters():
    param.requires_grad = True
###########

criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
criterion = criterion.to(device)
print("criterion: ", criterion)

temp = 5
alpha = 0.5


#def train(epochs=4, lr=1e-4):  # Change Epoch number to 30
    # '''
    # During Quantization Aware Training, the LR should be 1-10% of the LR used for training without quantization

    #optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr, momentum=0.9, dampening=0, weight_decay=1e-05)
   # optimizer = optim.Adam(
       # filter(lambda p: p.requires_grad, model.parameters()), lr, weight_decay=1e-5)

    # https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html
    # new_lr = lr * factor
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=2, threshold=0.01, min_lr=0.0001, verbose=True)
    # scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[7, 25], gamma=0.5, verbose=True)
    #scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
       # optimizer, T_0=15, T_mult=1, eta_min=1e-5, verbose=True)



teacher_model.eval()

# necessary variables
y_true = []
y_pred = []
decoded_preds = []
decoded_labels = []

total_wer = 0

#print("***Epoch: {}***".format(epoch))
batch_loss = 0

for i, (inp, img_names, idx) in enumerate(tqdm(train_loader)):
    inp = inp.to(device)
    #print("Input :", inp)
    # inp = inp.float()/255.
    batch_size = inp.size(0)
    idxs = idx.detach().numpy()
    img_names = list(img_names)
    words, labels, labels_size = get_padded_labels(
        idxs, words_tr, labels_tr, lengths_tr)

    # teacher prediction

    teacher_preds = teacher_model(inp)
    
    teacher_pred= rectify_tazin(teacher_preds,labels, inv_grapheme_dict)
    
   
    
  

    break
    
    
    
    

 13%|█████████▏                                                              | 11558/90000 [00:00<00:00, 114745.43it/s]


Preprocessing synthetic training data:


 26%|██████████████████▌                                                     | 23177/90000 [00:00<00:00, 111965.58it/s]


Preprocessing synthetic validation training data:


100%|████████████████████████████████████████████████████████████████████████| 17275/17275 [00:00<00:00, 116233.33it/s]



Preprocessing synthetic validation training data:


  0%|                                                                                          | 0/360 [00:00<?, ?it/s]

criterion:  CTCLoss()


  0%|                                                                                          | 0/360 [00:09<?, ?it/s]

[38  1  1  1  1  1  1  1  1  1  1]
<class 'numpy.ndarray'>
[16 20  1  1  1  1  1  1  1  1  1]
<class 'numpy.ndarray'>
[38  1  1  1  1  1  1  1  1  1  1]
<class 'numpy.ndarray'>
[38  1  1  1  1  1  1  1  1  1  1]
<class 'numpy.ndarray'>
[ 5  6  8 40 12 28  1  1  1  1  1]
<class 'numpy.ndarray'>
[34  4 17 20  4 36 33 11  1  1  1]
<class 'numpy.ndarray'>
[34  4 20 12 21 20  1  1  1  1  1]
<class 'numpy.ndarray'>
[ 5  4 40  3  2 13  1  1  1  1  1]
<class 'numpy.ndarray'>
[ 2 15 21  2 30 28 21 20  1  1  1]
<class 'numpy.ndarray'>
prediction [ 2  0  0  0  0  0  0 20  0 21 21  0  2  0  0 30  0  0  4  0 28  0 21  0
  0 20  0  0  0  0  0]
preds  [2, 20, 21, 2, 30, 4, 28, 21, 20]
labels [2, 15, 21, 2, 30, 28, 21, 20]
[20, 21, 28, 30, 2, 21, 15, 2]
--------------Recitified-----------
[ 2  0  0  0  0  0  0 15  0 21 21  0  2  0  0 30  0  0  0  0 28  0 21  0
  0 20  0  0  0  0  0]


[12  4 22 11  2  4 15  1  1  1  1]
<class 'numpy.ndarray'>
[30 28 11 12  1  1  1  1  1  1  1]
<class 'numpy.ndarray'>



## For pred> label

In [6]:
################################# All Diacritics Seperation #################################
def ads_grapheme_extraction(word):

    forms_cluster = {'ক': ['ক', 'ট', 'ত', 'ন', 'ব', 'ম', 'র', 'ল', 'ষ', 'স'],
                     'গ': ['গ', 'ধ', 'ন', 'ব', 'ম', 'ল'],
                     'ঘ': ['ন'],
                     'ঙ': ['ক', 'খ', 'গ', 'ঘ', 'ম'],
                     'চ': ['চ', 'ছ', 'ঞ'],
                     'জ': ['জ', 'ঝ', 'ঞ', 'ব'],
                     'ঞ': ['চ', 'ছ', 'জ', 'ঝ'],
                     'ট': ['ট', 'ব'],
                     'ড': ['ড'],
                     'ণ': ['ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ব', 'ম'],
                     'ত': ['ত', 'থ', 'ন', 'ব', 'ম', 'র'],
                     'থ': ['ব'],
                     'দ': ['গ', 'ঘ', 'দ', 'ধ', 'ব', 'ভ', 'ম'],
                     'ধ': ['ন', 'ব'],
                     'ন': ['জ', 'ট', 'ঠ', 'ড', 'ত', 'থ', 'দ', 'ধ', 'ন', 'ব', 'ম', 'স'],
                     'প': ['ট', 'ত', 'ন', 'প', 'ল', 'স'],
                     'ফ': ['ট', 'ল'],
                     'ব': ['জ', 'দ', 'ধ', 'ব', 'ভ', 'ল'],
                     'ভ': ['র'],
                     'ম': ['ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'ল'],
                     'ল': ['ক', 'গ', 'ট', 'ড', 'প', 'ফ', 'ব', 'ম', 'ল', 'স'],
                     'শ': ['চ', 'ছ', 'ত', 'ন', 'ব', 'ম', 'ল'],
                     'ষ': ['ক', 'ট', 'ঠ', 'ণ', 'প', 'ফ', 'ব', 'ম'],
                     'স': ['ক', 'খ', 'ট', 'ত', 'থ', 'ন', 'প', 'ফ', 'ব', 'ম', 'ল'],
                     'হ': ['ণ', 'ন', 'ব', 'ম', 'ল'],
                     'ড়': ['গ']}

    forms_tripple_cluster = {'ক্ষ': ['ণ', 'ম'], 'ঙ্ক': ['ষ'], 'চ্ছ': ['ব'], 'জ্জ': ['ব'],
                             'ত্ত': ['ব'], 'দ্দ': ['ব'], 'দ্ধ': ['ব'], 'দ্ভ': ['র'],
                             'ন্ত': ['ব'], 'ন্দ': ['ব'], 'ম্প': ['ল'], 'ম্ভ': ['র'],
                             'ষ্ক': ['র'], 'স্ক': ['র'], 'স্ত': ['ব', 'র'], 'স্প': ['ল']}

    chars = []
    i = 0
    adjust = 0

    while(i < len(word)):
        if i+1 < len(word) and word[i+1] == '্':
            if word[i] == 'র':
                chars.append('র্')
                adjust = 0
                i += 2
            elif i+2 < len(word) and word[i+2] == 'য':
                chars.append(word[i-adjust:i+1])
                chars.append('্য')
                adjust = 0
                i += 3
            elif i+2 < len(word) and word[i+2] == 'র':
                # Treat '্র' as a seperate grapheme
                chars.append(word[i-adjust:i+1])
                chars.append('্র')
                # Keep '্র' icluded in the cluster
                # chars.append(word[i-adjust:i+3])
                if i+3 < len(word) and word[i+3] == '্' and i+4 < len(word) and word[i+4] == 'য':
                    chars.append('্য')
                    i += 5
                else:
                    i += 3
                adjust = 0
            elif i+2 < len(word) and adjust != 0 and word[i-adjust:i+1] in forms_tripple_cluster \
                    and word[i+2] in forms_tripple_cluster[word[i-adjust:i+1]]:
                if i+3 < len(word) and word[i+3] == '্':
                    adjust += 2
                    i += 2
                else:
                    chars.append(word[i-adjust:i+3])
                    adjust = 0
                    i += 3
            elif i+2 < len(word) and adjust == 0 and word[i] in forms_cluster and word[i+2] in forms_cluster[word[i]]:
                if i+3 < len(word) and word[i+3] == '্':
                    adjust += 2
                    i += 2
                else:
                    chars.append(word[i-adjust:i+3])
                    adjust = 0
                    i += 3
            else:
                chars.append(word[i-adjust:i+1])
                chars.append('্')
                adjust = 0
                i += 2

        else:
            chars.append(word[i:i+1])
            i += 1

    # print(word)
    # print(chars)

    return chars


################################# Vowel Diacritics Seperation #################################
def vds_grapheme_extraction(word):

    consonants = ['ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ',
                  'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', 'ড়', 'ঢ়', 'য়']

    forms_cluster = {'ক': ['ক', 'ট', 'ত', 'ন', 'ব', 'ম', 'র', 'ল', 'ষ', 'স'],
                     'খ': ['র'],
                     'গ': ['গ', 'ধ', 'ন', 'ব', 'ম', 'র', 'ল'],
                     'ঘ': ['ন', 'র'],
                     'ঙ': ['ক', 'খ', 'গ', 'ঘ', 'ম', 'র'],
                     'চ': ['চ', 'ছ', 'ঞ', 'র'],
                     'ছ': ['র'],
                     'জ': ['জ', 'ঝ', 'ঞ', 'ব', 'র'],
                     'ঝ': ['র'],
                     'ঞ': ['চ', 'ছ', 'জ', 'ঝ', 'র'],
                     'ট': ['ট', 'ব', 'র'],
                     'ঠ': ['র'],
                     'ড': ['ড', 'র'],
                     'ঢ': ['র'],
                     'ণ': ['ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ব', 'ম', 'র'],
                     'ত': ['ত', 'থ', 'ন', 'ব', 'ম', 'র'],
                     'থ': ['ব', 'র'],
                     'দ': ['গ', 'ঘ', 'দ', 'ধ', 'ব', 'ভ', 'ম', 'র'],
                     'ধ': ['ন', 'ব', 'র'],
                     'ন': ['জ', 'ট', 'ঠ', 'ড', 'ত', 'থ', 'দ', 'ধ', 'ন', 'ব', 'ম', 'র', 'স'],
                     'প': ['ট', 'ত', 'ন', 'প', 'ল', 'র', 'স'],
                     'ফ': ['ট', 'র', 'ল'],
                     'ব': ['জ', 'দ', 'ধ', 'ব', 'ভ', 'র', 'ল'],
                     'ভ': ['র'],
                     'ম': ['ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'র', 'ল'],
                     'য': ['র'],
                     'র': ['ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ',
                           'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', 'ড়', 'ঢ়', 'য়'],
                     'ল': ['ক', 'গ', 'ট', 'ড', 'প', 'ফ', 'ব', 'ম', 'র', 'ল', 'স'],
                     'শ': ['চ', 'ছ', 'ত', 'ন', 'ব', 'ম', 'র', 'ল'],
                     'ষ': ['ক', 'ট', 'ঠ', 'ণ', 'প', 'ফ', 'ব', 'ম', 'র'],
                     'স': ['ক', 'খ', 'ট', 'ত', 'থ', 'ন', 'প', 'ফ', 'ব', 'ম', 'র', 'ল'],
                     'হ': ['ণ', 'ন', 'ব', 'ম', 'র', 'ল'],
                     'ড়': ['গ', 'র'],
                     'ঢ়': ['র'],
                     'য়': ['র']}

    forms_tripple_cluster = {'ক্ট': ['র'], 'ক্ত': ['র'], 'ক্ষ': ['ণ', 'ম', 'র'], 'ঙ্ক': ['ষ', 'র'], 'চ্ছ': ['ব', 'র'], 'জ্জ': ['ব'],
                             'ণ্ড': ['র'], 'ত্ত': ['ব'], 'দ্দ': ['ব'], 'দ্ধ': ['ব'], 'দ্ভ': ['র'], 'ন্ট': ['র'], 'ন্ড': ['র'], 'ন্ত': ['ব', 'র'],
                             'ন্দ': ['ব', 'র'], 'ন্ধ': ['র'], 'ম্প': ['র', 'ল'], 'ম্ভ': ['র'],
                             'ষ্ক': ['র'], 'স্ক': ['র'], 'ষ্ট': ['র'], 'স্ট': ['র'], 'স্ত': ['ব', 'র'], 'ষ্প': ['র'], 'স্প': ['র', 'ল'],
                             # refs
                             'র্ক': ['র', 'ট', 'ত', 'ল', 'ষ', 'স'], 'র্খ': ['র'], 'র্গ': ['ব', 'ল', 'র'], 'র্ঘ': ['র'], 'র্ঙ': ['ক', 'গ', 'র'],
                             'র্চ': ['চ', 'ছ', 'র'], 'র্ছ': ['র'], 'র্জ': ['জ', 'ঞ', 'র'], 'র্ঝ': ['র'], 'র্ঞ': ['জ', 'র'],
                             'র্ট': ['ট', 'ম', 'র'], 'র্ঠ': ['র'], 'র্ড': ['র'], 'র্ঢ': ['র'], 'র্ণ': ['ড', 'ন', 'র'],
                             'র্ত': ['ত', 'থ', 'ন', 'ব', 'ম', 'র'], 'র্থ': ['র'], 'র্দ': ['জ', 'থ', 'দ', 'ধ', 'ব', 'র'], 'র্ধ': ['ব', 'ম', 'র'],
                             'র্ন': ['ট', 'ড', 'ত', 'দ', 'ন', 'ব', 'ম', 'স', 'র'],
                             'র্প': ['ক', 'প', 'স', 'র'], 'র্ফ': ['র'], 'র্ব': ['জ', 'ব', 'ল', 'র'], 'র্ভ': ['র'], 'র্ম': ['প', 'ব', 'ম', 'র'],
                             'র্য': ['র'], 'র্র': ['র'], 'র্ল': ['র', 'ট', 'ড', 'স'], 'র্শ': ['চ', 'ন', 'ব', 'র'], 'র্ষ': ['ক', 'ট', 'ণ', 'প', 'ম', 'র'],
                             'র্স': ['ক', 'চ', 'ট', 'ত', 'থ', 'প', 'ফ', 'ব', 'ম', 'র'], 'র্হ': ['র'], 'র্ড়': ['র'], 'র্ঢ়': ['র'], 'র্য়': ['র']}

    chars = []
    i = 0
    adjust = 0

    while(i < len(word)):
        if i+1 < len(word) and word[i+1] == '্':
            if i+2 < len(word) and word[i+2] == 'য':
                if word[i] in consonants:
                    chars.append(word[i-adjust:i+3])
                else:
                    chars.append(word[i-adjust:i+1])
                    chars.append('্য')
                adjust = 0
                i += 3
            elif i+2 < len(word) and adjust != 0 and word[i-adjust:i+1] in forms_tripple_cluster \
                    and word[i+2] in forms_tripple_cluster[word[i-adjust:i+1]]:
                if i+3 < len(word) and word[i+3] == '্':
                    adjust += 2
                    i += 2
                else:
                    chars.append(word[i-adjust:i+3])
                    adjust = 0
                    i += 3
            elif i+2 < len(word) and adjust == 0 and word[i] in forms_cluster and word[i+2] in forms_cluster[word[i]]:
                if i+3 < len(word) and word[i+3] == '্':
                    adjust += 2
                    i += 2
                else:
                    chars.append(word[i-adjust:i+3])
                    adjust = 0
                    i += 3
            else:
                chars.append(word[i-adjust:i+1])
                chars.append('্')
                adjust = 0
                i += 2

        else:
            chars.append(word[i:i+1])
            i += 1

    # print(word)
    # print(chars)

    return chars


################################# Naive character representation #################################
def naive_grapheme_extraction(word):

    return list(word)


In [7]:
def nextIndex(arr, index):
    next_index= 0
    previous = 0
    l = 0
    
    for i in range(index, len(arr)):
        if(arr[i]==0):    
            continue
        elif(arr[i] == previous):
            continue
        else: 
            previous = arr[i]
            if(l == 0):
                l = 1
                continue
            elif(l==1):
                return arr[i]
                


# if found not match 
# just check next case if match make the existing one = 0
# if the next case doesnt match replace label with pred. 

In [8]:
# test pred now contains value only those which are present in label tensor

# now check if test pred > label or test_pred = label or test_pred < label

# if test_pred > label still use tazin

# if test_pred > label still use replace_pred

# if test_pred < label still use getPred(nijhum bhai)



# old ->  [28  0  0  0  0  4  4  0 12  0  0  4  4 62  0  0  7  0  0  0 21  0  0  0
#   0 20  0  0  0  0  0]
# pred :  [28, 4, 12, 4, 62, 7, 21, 20]
# label [7, 12, 4, 62, 7, 21, 20]


# old ->  [53  0  0  0  0  0  0  0  0  0 21  0  0  0  0  0 12  0  0  0  0  0  4  0
#   0  0  0  0  0  0  0]
# pred :  [53, 21, 12, 4]
# label [53, 21, 12]


# old ->  [ 2  0  0  0  0  0  0 20  0 21 21  0  2  0  0 30  0  0  4  0 28  0 21  0
#   0 20  0  0  0  0  0]
# pred :  [2, 20, 21, 2, 30, 4, 28, 21, 20]
# label [2, 15, 21, 2, 30, 28, 21, 20]


# old ->  [30  0  0  0  0  0  0  0  4  0 12  0  0  0 40  8  0  0  0 40 14  0  0  0
#   0  0  0  0  0  0  0]
# pred :  [30, 4, 12, 40, 8, 40, 14]
# label [30, 4, 12, 46, 40, 14

# old ->  [53  0  0  0  0  0  0  0  0  0 21  0  0  0  0  0 12  0  0  0  0  0  4  0
#   0  0  0  0  0  0  0]
# pred :  [53, 21, 12, 4]
# label [53, 21, 12]

import numpy as np 
test_pred = np.array( [53,  0,  0 , 0 , 0 , 0 , 0  ,0  ,0  ,0 ,21,  0,  0,  0 , 0,  0, 12 , 0  ,0 , 0 , 0 , 0,  4,  0,
   0 , 0,  0,  0 , 0,  0 , 0])

test_label = np.array([53, 21, 12])
test_label = test_label[::-1]
test_label = test_label.tolist()

l = 0
x = 0 
prev= 0

store_value_from_label  = 0


new_pred = []
store_index = 0 
for i in range(len(test_pred)):

    if(test_pred[i]==0):
        continue
    elif(test_pred[i] == x):
        continue

    else:
       
        store_index = i    
        x = test_pred[i]
        next_index = nextIndex(test_pred, i)
        
        
    
       
        if(test_label):
            popped_from_stack = test_label.pop()
        else:
            popped_from_stack = 0 
        
        print('popped : ', popped_from_stack)
        
       

        
        if(popped_from_stack != x): 
            if(popped_from_stack == next_index):
                test_label.append(popped_from_stack)
                print(popped_from_stack, "-- > ", store_index , " -> ", next_index)  
                test_pred[store_index] = 0 
                
                if(test_pred[i+1] == x):
                    test_pred[i+1] = 0 
                    
                print(test_pred)
                print('\n')

            elif(popped_from_stack!= next_index):
                test_pred[i] = popped_from_stack 
                
  
                
print(test_pred)   



popped :  53
popped :  21
popped :  12
popped :  0
[53  0  0  0  0  0  0  0  0  0 21  0  0  0  0  0 12  0  0  0  0  0  0  0
  0  0  0  0  0  0  0]


In [9]:
# old ->  [30  0  0  0  0  0  0  0  4  0 12  0  0  0 40  8  0  0  0 40 14  0  0  0
#   0  0  0  0  0  0  0]
# pred :  [30, 4, 12, 40, 8, 40, 14]
# label [30, 4, 12, 46, 40, 14]

In [10]:
len(new_pred)

0

In [11]:
test_pred_decoded = np.array([28, 4, 12, 4, 62, 7, 21, 20])
test_label = np.array([7, 12, 4, 62, 7, 21, 20])
test_label = test_label[::-1]
test_label = test_label.tolist()

In [12]:
print(test_label)
a = test_label.pop()

[20, 21, 7, 62, 4, 12, 7]


In [13]:
test_label.append(a)

In [14]:
test_ab = np.array([ 5,  0,  6,  6,  0,  0,  0,  0, 15,  0 ,21,  0 , 0 , 0, 25 , 0 , 0 , 0,  0,  0 , 0, 28,  0, 21,
 21 , 0 ,20,  0,  0 , 0,  0])
decoded, _ = decode_prediction(test_ab, inv_grapheme_dict)

test_ac = np.array([5, 6, 15, 35, 30, 28, 21, 20])

print(decoded)
print(test_ac)

[5, 6, 15, 21, 25, 28, 21, 20]
[ 5  6 15 35 30 28 21 20]


# Setting


In [15]:


import numpy as np 



test_ab = np.array([11,  0,  0,  0 , 0 , 0 , 0 , 0,  0, 10,  0,  0,  0,  0,  0,  0, 28, 23,  0 , 0 , 0, 23, 
 0,  3, 0,  0 , 0,  0,  0])

test_ac = np.array([11, 30, 28, 23, 3])


# test_ab = np.array([11,  0 , 0 , 0 , 0,  0 , 0,  0,  0 , 0, 46 , 4, 4 , 0 , 0 , 0 , 0,  0,  0 ,14,  9 , 9,  0,  8,
#  13 , 0,  0,  0 , 0])

# test_ac = np.array([11, 46, 4, 14, 13])





def nextIndex(arr, index):  # get value of next index
    previous = 0
    l = 0

    for i in range(index, len(arr)):
        if(arr[i] == 0):
            continue
        elif(arr[i] == previous):
            continue
        else:
            previous = arr[i]
            if(l == 0):
                l = 1
                continue
            elif(l == 1):
                return arr[i]


def predGreaterLabel(test_pred, test_label):
    pop = 0 
    x = 0
    

    test_label = test_label[::-1]
    test_label = list(test_label)
    print(test_label)

    for i in range(len(test_pred)):
        
        
        if(test_pred[i] == 0):  # ignore all 0's
            x = test_pred[i]
            continue
        elif(test_pred[i] == x):  # ignore same values < 4,4 >
            if(popped_from_stack == 0):
                test_pred[i] = 0
            try:
                if(test_pred[i-1] == popped_from_stack):
                    test_pred[i] = popped_from_stack
            except:
                print("safe to ignore")
                
                
            continue

        else:

            
            x = test_pred[i]
            next_index = nextIndex(test_pred, i)
            
            if(test_label):
                popped_from_stack = test_label.pop()
            else:
                popped_from_stack = 0
            
            if(popped_from_stack == 0):
                test_pred[i] = 0

            elif(popped_from_stack != x):
                
                if(popped_from_stack == next_index):
                    test_label.append(popped_from_stack)
                    #print(popped_from_stack, "-- > ", i , " -> ", next_index)
                    test_pred[i] = 0
                    
                    
                    try:
                        if(test_pred[i+1] == x):
                            test_pred[i+1] = 0
                    except:
                        print('safe to ignore')

                elif(popped_from_stack != next_index):

                    test_pred[i] = popped_from_stack
                                  
                    



    return test_pred




ho = predGreaterLabel(test_ab, test_ac)
print(ho)   




# test_b = np.array([ 8,  9, 3 , 0 , 0,  0,  0,  3 , 0, 23 , 0 , 0,  0 ,54 , 9 , 9 , 3 , 3 , 3 ,37, 37 , 0,  0,  0,
#  14,  0 , 0,  0 , 0])

# test_aa = np.array([8, 23, 54, 9, 3, 37, 14])


# _, decoded_pred = decode_prediction(test_aa, inv_grapheme_dict)
# xd, _ = decode_prediction(test_b , inv_grapheme_dict)
# _, decoded_prec = decode_prediction(xd , inv_grapheme_dict)

# print(decoded_pred)
# decoded_prec

[3, 23, 28, 30, 11]
[11  0  0  0  0  0  0  0  0 30  0  0  0  0  0  0 28 23  0  0  0  0  0  3
  0  0  0  0  0]


In [16]:
#sad cases

[ 8  9  3  0  0  0  0  3  0 23  0  0  0 54  9  9  3  3  3 37 37  0  0  0
 14  0  0  0  0]

[ 8 23 54  0  0  0  0  3  0  9  0  0  0  3 37  9 14  3  3  0 37  0  0  0
  0  0  0  0  0]


[8, 23, 54, 9, 3, 37, 14]





[11  0  0  0  0  0  0  0  0  0 46  4  4  0  0  0  0  0  0 14  9  9  0  8
 13  0  0  0  0]


[11  0  0  0  0  0  0  0  0  0 46  4  4  0  0  0  0  0  0 14 13  9  0  0
  0  0  0  0  0]
[11, 46, 4, 14, 13]



SyntaxError: invalid syntax (<ipython-input-16-c5f6fde58eec>, line 3)

In [17]:
#sad cases

[ 8  9  3  0  0  0  0  3  0 23  0  0  0 54  9  9  3  3  3 37 37  0  0  0
 14  0  0  0  0]

[ 8 23 54  0  0  0  0  3  0  9  0  0  0  3 37  9 14  3  3  0 37  0  0  0
  0  0  0  0  0]


[8, 23, 54, 9, 3, 37, 14]


# sad cases 2 
[28  4  0  0  0  0  0  0  0  0  0 33  0  9 41 41 23 23  0  0  0  0  0  0
  0  0  0  0  0]

[28  4  0  0  0  0  0  0  0  0  0 41  0 23  0 41  0 23  0  0  0  0  0  0
  0  0  0  0  0]


[28, 4, 41, 23]

SyntaxError: invalid syntax (<ipython-input-17-681217270e00>, line 3)